In [2]:
import pandas as pd 
import seaborn as sns 
import sqlite3
import numpy as np
from plotly import express as px

with sqlite3.connect("../tickets.db") as conn: 
    cmd = \
        """
        SELECT g.*, e.price, e.extractTime, e.ticket_count FROM extraction e
        LEFT JOIN games g on e.id = g.id
        """
    df = pd.read_sql_query(cmd, conn)
df['time'], df['extractTime'] = pd.to_datetime(df['time']), pd.to_datetime(df['extractTime'])
df['days_to_game'] = (df['time'] - df['extractTime'])
df['days_to_game'] = df['days_to_game'].apply(lambda X: X.days)
df = df.dropna()
def diff_from_max(x):
    initial = x.to_numpy()[0]
    return (initial - x)/x
df["diff_initial"] = df.groupby("unique_name")["price"].transform(diff_from_max)
dodgers = df[df['homeTeam'] == 'Los Angeles Dodgers']
giants = df[df['homeTeam'] == 'San Francisco Giants']
angels = df[df['homeTeam'] == 'Los Angeles Angels']
df

,id,time,name,homeTeam,awayTeam,unique_name,price,extractTime,ticket_count,days_to_game,diff_initial
0,104722879.0,2021-06-11,Texas Rangers at Los Angeles Dodgers,Los Angeles Dodgers,Texas Rangers,Texas Rangers at Los Angeles Dodgers: 2021-06-11,58.62,2021-06-08 17:51:48.284897,0,2.0,0.000000
1,104725408.0,2021-06-12,Texas Rangers at Los Angeles Dodgers,Los Angeles Dodgers,Texas Rangers,Texas Rangers at Los Angeles Dodgers: 2021-06-12,29.16,2021-06-08 17:51:48.284897,0,3.0,0.000000
2,104725409.0,2021-06-13,Texas Rangers at Los Angeles Dodgers,Los Angeles Dodgers,Texas Rangers,Texas Rangers at Los Angeles Dodgers: 2021-06-13,28.04,2021-06-08 17:51:48.284897,0,4.0,0.000000
3,104724531.0,2021-06-14,Philadelphia Phillies at Los Angeles Dodgers,Los Angeles Dodgers,Philadelphia Phillies,Philadelphia Phillies at Los Angeles Dodgers: ...,35.55,2021-06-08 17:51:48.284897,0,5.0,0.000000
4,104725410.0,2021-06-15,Philadelphia Phillies at Los Angeles Dodgers,Los Angeles Dodgers,Philadelphia Phillies,Philadelphia Phillies at Los Angeles Dodgers: ...,24.99,2021-06-08 17:51:48.284897,0,6.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
9594,104738380.0,2021-09-29,Arizona Diamondbacks at San Francisco Giants,San Francisco Giants,Arizona Diamondbacks,Arizona Diamondbacks at San Francisco Giants: ...,20.41,2021-07-14 00:42:27.265461,2693,76.0,-0.158746
9595,104736599.0,2021-09-30,Arizona Diamondbacks at San Francisco Giants,San Francisco Giants,Arizona Diamondbacks,Arizona Diamondbacks at San Francisco Giants: ...,21.64,2021-07-14 00:42:27.265464,2666,77.0,-0.206562
9596,104736601.0,2021-10-01,San Diego Padres at San Francisco Giants,San Francisco Giants,San Diego Padres,San Diego Padres at San Francisco Giants: 2021...,33.92,2021-07-14 00:42:27.265467,2825,78.0,0.517099
9597,104738382.0,2021-10-02,San Diego Padres at San Francisco Giants,San Francisco Giants,San Diego Padres,San Diego Padres at San Francisco Giants: 2021...,54.76,2021-07-14 00:42:27.265470,2940,79.0,0.202155


# Dbacks vs Dodgers 7/11/21

In [3]:
gm1 = dodgers[dodgers["id"] == 104724575.0]
fig = px.line(gm1, x = "extractTime", y = "price", hover_name="time")
fig.update_traces(line_color='#456987')
# px.bar(gm1, x = "extractTime", y = "ticket_count")

In [4]:
px.bar(gm1, x = "extractTime", y = "ticket_count")

# Mariners vs. Angels 7/16/21

In [5]:
gm2 = angels[angels["id"] == 104731803.0]
px.line(gm2, x = "extractTime", y = "price")

In [5]:
px.bar(gm2, x = "extractTime", y = "ticket_count")

In [6]:
#

# Dodgers vs Giants 7/27/21

In [6]:
possible_gms = giants[giants["id"].isin([104738361.0, 104736558.0, 104736560.0])]
gm3 = giants[giants["id"] == 104738361.0]
fig = px.line(possible_gms, x = "extractTime", y = "price", facet_row= "id", height= 700, hover_name="time")
fig.update_traces(line_color='#F4793E')

In [8]:
px.bar(possible_gms, x = "extractTime", y = "ticket_count", facet_row= "id", height= 500)

# Week of 19 to 25

In [9]:
giants_gms = [104724576.0, 104724577.0, 104725433.0, 104724579.0]
rockies_gms = [104724581.0, 104724583.0, 104724585.0]
possible_gms = dodgers[dodgers["id"].isin(giants_gms)]
fig = px.line(possible_gms, x = "extractTime", y = "price", facet_row= "id", height= 700)
fig.update_traces(line_color='#F4793E')
fig.show()
fig2 = px.bar(possible_gms, x = "extractTime", y = "ticket_count", facet_row= "id", height= 500)
fig2.show()

In [7]:
possible_gms = dodgers[dodgers["id"].isin(rockies_gms)]
fig = px.line(possible_gms, x = "extractTime", y = "price", facet_row= "id", height= 700, hover_name="time")
fig.update_traces(line_color='#33006F')
fig.show()
fig2 = px.bar(possible_gms, x = "extractTime", y = "ticket_count", facet_row= "id", height= 500)
fig2.show()

NameError: name 'rockies_gms' is not defined

In [11]:
?px.line

Signature:
px.line(
    data_frame=None,
    x=None,
    y=None,
    line_group=None,
    color=None,
    line_dash=None,
    hover_name=None,
    hover_data=None,
    custom_data=None,
    text=None,
    facet_row=None,
    facet_col=None,
    facet_col_wrap=0,
    facet_row_spacing=None,
    facet_col_spacing=None,
    error_x=None,
    error_x_minus=None,
    error_y=None,
    error_y_minus=None,
    animation_frame=None,
    animation_group=None,
    category_orders=None,
    labels=None,
    orientation=None,
    color_discrete_sequence=None,
    color_discrete_map=None,
    line_dash_sequence=None,
    line_dash_map=None,
    log_x=False,
    log_y=False,
    range_x=None,
    range_y=None,
    line_shape=None,
    render_mode='auto',
    title=None,
    template=None,
    width=None,
    height=None,
)
Docstring:
    In a 2D line plot, each row of `data_frame` is represented as vertex of
    a polyline mark in 2D space.
    
Parameters
----------
data_frame: DataFrame or array-l